In [18]:
import pandas as pd
import numpy as np
import os
from dshortcuts.utils import open_pickle, save_pickle
# from operator import itemgetter
import geocoder

In [2]:
os.listdir('../input')

['test.csv',
 'sample_submission.csv',
 'item_categories.csv',
 'trainDtypes.pickle',
 'sales_train.csv',
 'itemCategoriesDtypes.pickle',
 'items.csv',
 'shops.csv']

In [3]:
# open prepared dtypes to save memory
trainDtypes = open_pickle(os.path.join('../input', 'trainDtypes.pickle'))
itemCategoriesDtypes = open_pickle(os.path.join('../input', 'itemCategoriesDtypes.pickle'))

In [4]:
train = pd.read_csv('../input/sales_train.csv', dtype=trainDtypes, parse_dates=['date'])
itemCategories = pd.read_csv('../input/item_categories.csv', dtype=itemCategoriesDtypes)
test = pd.read_csv('../input/test.csv')
shops = pd.read_csv('../input/shops.csv')
items = pd.read_csv('../input/items.csv')

In [5]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.000000,1.0
1,2013-03-01,0,25,2552,899.000000,1.0
2,2013-05-01,0,25,2552,899.000000,-1.0
3,2013-06-01,0,25,2554,1709.050049,1.0
4,2013-01-15,0,25,2555,1099.000000,1.0


##### Categories
1. Are they broad enough?
2. Are they descriptive enough?

In [6]:
itemCategories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


too descriptive. At first look
we may try to decouple broad categories by split('-')`

In [7]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [8]:
train = train.merge(items.drop('item_name', axis=1), on='item_id', how='left')

#### Obvious features: 

- ~~day~~
- month
- quarter
- year
- ~~dayofweek~~
- ~~is workingday~~
- ~~delta from every 22th day of month (~salary day in Rus)~~
- ~~delta from every 7th day of month (~pre-salary day in Rus)~~

I considered that in test set there is only a month that we know from a datetime features. It means that we can get only month+ features

In [9]:
# train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['quarter'] = train['date'].dt.quarter
# train['dayofweek'] = train['date'].dt.weekday
# train['is_weekend'] = (train['dayofweek'] >= 5).astype('int8')

In [10]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,month,year,quarter
0,2013-02-01,0,59,22154,999.000000,1.0,37,2,2013,1
1,2013-03-01,0,25,2552,899.000000,1.0,58,3,2013,1
2,2013-05-01,0,25,2552,899.000000,-1.0,58,5,2013,2
3,2013-06-01,0,25,2554,1709.050049,1.0,58,6,2013,2
4,2013-01-15,0,25,2555,1099.000000,1.0,56,1,2013,1


In [ ]:
..quantile(.75)

#### Sales & Price features: (think more...)

**H1**:
! People buy things when the price goes down

- 1quart price in previous month (shop-item)
- 3quart price in previous month (shop-item)
- price delta in prev month (shop-item)
- price delta over 2 months (shop-item)
- monthly sales by {shop_id, category_id}
- monthly sales by {shop_id, item_id}
- monthly sales by {category_id}

Probably it worth to make a **H2**:
! People buy when a shop signals about sales (no matter which item)

In [31]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_
shop_item = train.groupby(['date_block_num', 'shop_id', 'item_id'])

In [32]:
q25 = shop_item.item_price.quantile(.25)
q75 = shop_item.item_price.quantile(.75)

In [33]:
q25.reset_index()

,date_block_num,shop_id,item_id,item_price
0,0,0,32,221.00
1,0,0,33,347.00
2,0,0,35,247.00
3,0,0,43,221.00
4,0,0,51,127.75
...,...,...,...,...
1609119,33,59,22087,119.00
1609120,33,59,22088,119.00
1609121,33,59,22091,179.00
1609122,33,59,22100,629.00


In [39]:
train.shape, q25.shape

((2935849, 10), (1609124,))

In [23]:
shop_item.item_price.quantile(.75)

shop_id  item_id
0        30         265.0
         31         434.0
         32         221.0
         33         347.0
         35         247.0
                    ...  
59       22154      999.0
         22155      149.0
         22162      399.0
         22164      749.0
         22167      299.0
Name: item_price, Length: 424124, dtype: float64

In [26]:
((shop_item.item_price.quantile(.25) - shop_item.item_price.quantile(.75)) != 0)

shop_id  item_id
0        30         False
         31         False
         32         False
         33         False
         35         False
                    ...  
59       22154      False
         22155      False
         22162      False
         22164       True
         22167      False
Name: item_price, Length: 424124, dtype: bool

#### Geographical features
1. Get the city from the shop info
2. Try to get the lat lon of the shop with map api ?

-------------

- what's next?

In [ ]:
train.groupby(['date_block_num', 'shop_id', 'item_id']).item_cnt_day.sum()

In [ ]:
valid = train[train['date_block_num'] == (train['date_block_num'].max())]

In [ ]:
(test['shop_id'].isin(train['shop_id']) == False).sum(), \
(test['item_id'].isin(train['item_id']) == False).sum()

In [ ]:
test.head()